In [35]:
import pysam
import numpy as np
from collections import Counter
vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
          dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
          'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)

# input file is the reference name, reference based positions and mutations and type

samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/test/test.bam", "rb")

# TODO
# add in procedures for indels
# add in better output format, make it a matrix
# add in reference sequence so can determine more info later


phase_out = []
for chrom in set(vant["CHROM"]):
    seq_out = []
    pos = vant["POS"][vant["CHROM"] == chrom]
    call = vant["ALT"][vant["CHROM"] == chrom]
    ref = vant["REF"][vant["CHROM"] == chrom]
    try:
        for read in samfile.fetch(chrom, until_eof=True):
            try:
                #convert to positions in the read
                pos_conv = read.query_alignment_start + read.get_reference_positions()[0] #unaligned leading bases + position of first alignment on reference
                pos_conv = pos - pos_conv
                phase = ""
                call_c = 0
                for p in pos_conv:
                    try:
                        # we dont want any bases that were filtered out so only take those that match the input
                        if read.query_sequence[p] in call[call_c].split(',') or read.query_sequence[p] == ref[call_c]: # in case more than on allele
                            phase += read.query_sequence[p]
                        else:
                            phase += "N"
                    except Exception:
                        phase += "N"
                    call_c += 1
                seq_out.append(phase)
            except Exception:
                continue
        # phase_out.append([[chrom], [Counter(seq_out)]])
        phase_out.append([[chrom], list(Counter(seq_out).items())])
    except Exception:
        print(chrom, " not found in bam file!")

rpoC  not found in bam file!


In [ ]:
phase_out